In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import numpy as np
import random
import os 
import torch
from random import randint
# from utils.loss_utils import l1_loss
from gaussian_renderer import render_flow as render
import sys
stdout = sys.stdout
from scene import  Scene
from scene.flexible_deform_model import TissueGaussianModel
# from scene.tool_movement_model import GaussianModelActor
from scene.mis_gaussian_model import MisGaussianModel
from utils.general_utils import safe_state
from tqdm import tqdm
from utils.image_utils import psnr
from argparse import ArgumentParser, Namespace
from arguments import ModelParams, PipelineParams, OptimizationParams
from arguments import FDMHiddenParams as ModelHiddenParams
from utils.timer import Timer
import torch.nn.functional as F
# import lpips
from utils.scene_utils import render_training_image
from scene.cameras import Camera
from utils.loss_utils import ssim
from train import training_report
from utils.image_utils import save_img_torch, visualize_depth_numpy
from datetime import datetime

from train import setup_seed
from train_utils_misgs import training_misgsmodel,scene_reconstruction_misgs,training_report_misgs
import argparse

parser = ArgumentParser(description="Training script parameters")
setup_seed(6666)
lp = ModelParams(parser)
op = OptimizationParams(parser)
pp = PipelineParams(parser)
hp = ModelHiddenParams(parser)
parser.add_argument('--ip', type=str, default="127.0.0.1")
parser.add_argument('--port', type=int, default=6009)
parser.add_argument('--debug_from', type=int, default=-1)
parser.add_argument('--detect_anomaly', action='store_true', default=False)
parser.add_argument("--test_iterations", nargs="+", type=int, default=[i*500 for i in range(0,120)])
parser.add_argument("--save_iterations", nargs="+", type=int, default=[0,1])
parser.add_argument("--checkpoint_iterations", nargs="+", type=int, default=[1])
parser.add_argument("--quiet", action="store_true")
parser.add_argument("--start_checkpoint", type=str, default = None)
parser.add_argument("--expname", type=str, default = "endonerf/pulling_fdm")
parser.add_argument("--configs", type=str, default = "arguments/endonerf/default.py")
# args = parser.parse_args(sys.argv[1:])
args = parser.parse_args("")


try:
    from torch.utils.tensorboard import SummaryWriter
    TENSORBOARD_FOUND = True
except ImportError:
    TENSORBOARD_FOUND = False

In [ ]:
#////////////////////////////////////////////
#use misgs env python
mode='P2_7_170_195' #'pulling''cutting' #'P2_7_170_195'
METHOD='deform3dgs' #'surggs'#'endog' 'deform3dgs'
EXP_APPEND='_finaldbg'
#other related
ITER_TIMES = 2
TENSORBOARD_FOUND = False
TENSORBOARD_FOUND = True

#////////////////////////////////////////////
use_stree_grouping_strategy = True
# use_stree_grouping_strategy = False
if use_stree_grouping_strategy:
    use_streetgs_render = True #fail
    use_streetgs_render = False
#///////////////////////////
if 'P2_' in mode:
    BASE_DATASET_PATH="/mnt/cluster/datasets/StereoMIS_processed"
    BASE_EXPERIMENT_PATH="/mnt/ceph/tco/TCO-Staff/Homes/jinjing/exps/train/gs/SM"
if mode in ['pulling','cutting']:
    BASE_DATASET_PATH="/mnt/cluster/datasets/EndoNeRF"
    BASE_EXPERIMENT_PATH="/mnt/ceph/tco/TCO-Staff/Homes/jinjing/exps/train/gs/EndoNeRF"
DATASET_PATH=f"{BASE_DATASET_PATH}/{mode}"
TIMESTAMP = datetime.now().strftime("%m-%d_%H-%M-%S")
TIMESTAMP = 'DBG_TIME'#datetime.now().strftime("%m-%d_%H-%M-%S")
EXPERIMENT_PATH=f"{BASE_EXPERIMENT_PATH}/{mode}/{METHOD}{EXP_APPEND}/{TIMESTAMP}"
CONFIG_PATH="/mnt/ceph/tco/TCO-Staff/Homes/jinjing/proj/gs/baselines/Deform3DGS/arguments/endonerf/default.py"
args._source_path = DATASET_PATH
args.source_path = DATASET_PATH
args.expname = EXPERIMENT_PATH
args.configs = CONFIG_PATH
#//////////update with config
assert args.configs!= None
import mmcv
from utils.params_utils import merge_hparams
config = mmcv.Config.fromfile(args.configs)
args = merge_hparams(args, config)
#update with tool_info automatically
expname_append = ''
if hasattr(args,'tool_mask'):
    expname_append += f'_{args.tool_mask}'
if hasattr(args,'init_mode'):
    expname_append += f'_{args.init_mode}'
setattr(args, 'expname', f'{args.expname}{expname_append}')
if 'pulling' in args.source_path or 'cutting' in args.source_path:
    pass
    print('TODO','nouse and inverse might be problematic---the data gt depth always masked tool region....')
    # assert args.tool_mask == "use",'nouse and inverse might be problematic---the data gt depth always masked tool region....'
args.save_iterations.append(args.iterations)
args.model_path = args.expname#jj--use by misgs
print("Optimizing " + args.model_path)
# Initialize system state (RNG)
safe_state(args.quiet)
print(args.iterations)
#//////////update with ours
args.iterations = ITER_TIMES
print('THE TEST Iters:',args.iterations)

In [ ]:
sys.stdout = stdout #this is important to avoid the no printout issue
args.save_iterations = []
#/////////////////////////////////
from train import training
from train_utils_misgs import training_misgsmodel
torch.cuda.empty_cache()
torch.autograd.set_detect_anomaly(args.detect_anomaly)
# All done
print("\nTraining start.", args.model_path)
if not use_stree_grouping_strategy:
    print('START SINGLE GS...')
    training(lp.extract(args), hp.extract(args), op.extract(args), pp.extract(args), args.test_iterations, \
        args.save_iterations, args.checkpoint_iterations, args.start_checkpoint, args.debug_from, args.expname, 
        extra_mark=args.extra_mark,
        args = args)
else:
    print('START MISGS...')
    from train_utils_misgs import training_misgsmodel
    training_misgsmodel(args,use_streetgs_render = use_streetgs_render)
# All done
print("\nTraining complete.", args.model_path)